# Simulation of Planetary Orbits

## Introduction

The following notebook incorporates animations that use Newton's Law of Universal Gravitation to simulate the orbit of planets around a star. First starting with only one planet, before simulating two planets, the first time ignoring the gravitational force between the two and the second time including the gravitational interaction. For these simulations the Euler numerical method was used to approximate the values of the position and velocity of the planets as they orbited the star. The gravitational force exerted on $m_1$ between two objects with masses $m_1$ and $m_2$ is [1]:
$$
\vec{F_{12}} = -G\frac{m_1 m_2}{|\vec{r}_{21}|^2} \hat{r}_{21}
$$
where $G$ is the gravitational constant, $\vec{F_{12}}$ is the force applied on mass 1 and $\vec{r}_{21}$ is the vector between mass 1 and mass 2.

Using this force we can calculate the acceleration by using $F =ma $ (Newton's Second Law) and thus apply Euler's numerical method which uses the following equations [1]:

$$ r(t+\delta t) = r(t) +v\delta t$$
$$ v(t+\delta t) = v(t) +\delta v $$
$$ \delta v = a \delta t = -\frac{GM\hat{r}}{|\vec{r}|^2} \delta t$$
where $r$ is the position as a function of time and $v$ is the velocity as a function of time and $\delta$ signifying a very small finite change in a quantity.

In Part A, the main goal is first creating appropriate functions to simulate a two-body system of a star and a planet orbiting the star. Part B focuses on adapting these functions to now work for 2 planets and investigating the effects of two planets in orbit around a star.


In [ ]:
# import libraries
%matplotlib notebook
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML
import numpy as np
from numpy.linalg import norm

## Part A

### 1. Calculating the gravitational force between two objects

You should complete the following function, without changing its name, arguments or docstring, to calculate the force on one object due to the gravitational field of another.

In the following code cell the force calculated uses a slightly different form of the force equation for gravity [1]:
$$ \vec{F} = -\frac{GMm\hat{r}}{|\vec{r}|^2} = -\frac{GMm\vec{r}}{|\vec{r}|^3} $$

This will make it simpler to code compared to the original as the unit vector won't have to be calculated.

In [ ]:
def force(pos1, pos2, m1, m2):
    """
    Returns the gravitational force exerted by object 2 on object 1.
    Input:
      - pos1 = position vector of first object  (NumPy array)
      - pos2 = position vector of second object (NumPy array)
      - m1   = mass of first object
      - m2   = mass of second object
    Depends on:
      - G    = gravitational constant (global variable)
    """
    ##################
    relpos = pos1 - pos2
    forceOn1 = (-1*G*m1*m2*relpos)/(np.linalg.norm(relpos)**3)
    return forceOn1
    ##################

#### Testing your function

The following cell applies some tests to help you make sure your `force` function works correctly before you use it in the rest of the task. You do not need to understand the details of how it works, but it may help you narrow down any bugs in your code. If each line of output starts with `OK` then it is likely (but not guaranteed) that you have implemented the function correctly.

Please leave the code in this cell unchanged: you may add your own tests if you wish, but these should be in a separate cell.

In [ ]:
################################################
#                                              #
# Test force is correct in a few simple cases. #
#                                              #
#   DO NOT CHANGE THE CODE IN THIS CELL.       #
#                                              #
################################################

def test_force(pos1, pos2, m1, m2, expected_force):
    """Check whether force function gives expected results."""
    epsilon = 1e-10
    f = force(np.array(pos1), np.array(pos2), m1, m2)
    if not isinstance(f,np.ndarray):
        print(f"ERROR: function should return a vector but returns {f}.")
        return
    args_as_string = f"({pos1}, {pos2}, {m1}, {m2})"
    error = norm(f-expected_force)
    if error<epsilon:
        print(f"OK: correct results for input {args_as_string}")
    else:
        print(f"ERROR: wrong results for input {args_as_string}")
        print(f"  expected: {expected_force}")
        print(f"  got:      {f}")

# Set global variable G to 1 for now to make testing easier
G=1

# Test force with some simple cases
test_force((0,0),(1,0),1,1,(1,0))    # distance = 1 in x direction
test_force((1,0),(0,0),1,1,(-1,0))   # swap objects
test_force((0,0),(2,0),1,1,(0.25,0)) # distance = 2
test_force((0,0),(0,1),1,1,(0,1))    # distance = 1 in y direction
test_force((10,0),(10,1),1,1,(0,1))  # displaced from origin
test_force((0,0),(1,0),2,1,(2,0))    # non-unit mass 1
test_force((0,0),(1,0),1,2,(2,0))    # non-unit mass 2

### 2. Calculating the motion of a planet in the gravitational field of a star

You should complete the following function, without changing its name, arguments or docstring, to calculate the new position and velocity of a planet after a time step `dt`, in the gravitational field of a star of a given mass situated at the origin. This function will need to call `force` to calculate the acceleration vector of the planet.

The acceleration calculated uses the following equation [1]:
$$ a = \frac{\vec{F}}{m} = -\frac{GM\vec{r}}{|\vec{r}|^3} $$
So when using the force function we can set the mass of the planet = 1, which will produce the equivalent acceleration.

In [ ]:
def move_planet(position, velocity, m_star, dt):
    """
    Calculate motion of planet in the gravitational field of a star with given mass
    at the origin, using Euler's method.
    
    Input:
      - position: position vector of planet at start of time step (NumPy array)
      - velocity: velocity vector of planet at start of time step (NumPy array)
      - m_star:   mass of star
      - dt:       time step
      
    Output: (position_new, velocity_new)
      - position_new: position vector of planet at end of time step (NumPy array)
      - velocity_new: velocity vector of planet at end of time step (NumPy array)
      
    Depends on:
      - force = function to calculate the gravitational force between two objects
    """
    ##################
    starpos = np.array([0,0])
    acceleration = force(position, starpos, 1, m_star)
    position_new = position + np.array(velocity) * dt
    velocity_new = np.array(velocity) + acceleration * dt
    return position_new, velocity_new
    ##################

#### Testing your function

The following cell applies some tests to help you make sure your `move_planet` function works correctly before you use it in the rest of the task. You do not need to understand the details of how it works, but it may help you narrow down any bugs in your code. If each line of output starts with `OK` then it is likely (but not guaranteed) that you have implemented the function correctly.

Please leave the code in this cell unchanged: you may add your own tests if you wish, but these should be in a separate cell. 

In [ ]:
######################################################
#                                                    #
# Test move_planet is correct in a few simple cases. #
#                                                    #
#   DO NOT CHANGE THE CODE IN THIS CELL.             #
#                                                    #
######################################################

def test_move_planet(position, velocity, m_star, dt, expected_pos, expected_vel):
    """Check whether move_planet function gives expected results."""
    epsilon = 1e-10
    results = move_planet(np.array(position), np.array(velocity), m_star, dt)
    if not isinstance(results, tuple):
        print(f"ERROR: function should return two vectors but returns {results}.")
        return
    if not len(results)==2:
        print(f"ERROR: function should return two vectors but returns {results}.")
        return
    pos, vel = results
    if not (isinstance(pos,np.ndarray) and isinstance(vel,np.ndarray)):
        print(f"ERROR: function should return two vectors but returns {results}.")
        return
    args_as_string = f"{position}, {velocity}, {m_star}, {dt}"
    err_pos, err_vel = norm(pos - expected_pos), norm(vel - expected_vel)
    if err_pos < epsilon and err_vel < epsilon:
        print(f"OK: correct results for input {args_as_string}")
    else:
        print(f"ERROR: wrong results for input {args_as_string}")
        print(f"  expected: {expected_pos, expected_vel}")
        print(f"  got:      {results}")

# Set global variable G to 1 for now to make testing easier
G=1

# Test move_planet with some simple cases
test_move_planet((1,0), (1,0), 1, 0, (1,0), (1,0))    # dt = 0: output = input
test_move_planet((1,0), (1,0), 1, 1, (2,0), (0,0))    # moving away from star
test_move_planet((0,1), (0,-1), 1, 1, (0,0), (0,-2))  # moving towards star
test_move_planet((1,0), (0,1), 1, 1, (1,1), (-1,1))   # moving past star
test_move_planet((1,0), (0,1), 1, 0.1, (1,0.1), (-0.1,1))  # smaller dt
test_move_planet((1,0), (0,1), 2, 0.1, (1,0.1), (-0.2,1))  # larger star mass
test_move_planet((2,0), (1,0), 1, 1, (3,0), (0.75,0)) # non-unit distance

### 3. Calculating the orbit of a planet

You should complete the following function, without changing its name, arguments or docstring, to calculate the $x$ and $y$ coordinates of a planet at each time step.

In [ ]:
def trajectory(position, velocity, m_star, dt, t_max):
    """
    Calculate trajectory of planet from given starting position, using Euler's method.
    
    Input:
      - position: position vector of planet at start of simulation [m] (NumPy array)
      - velocity: velocity vector of planet at start of simulation [m] (NumPy array)
      - m_star:   mass of star [kg]
      - dt:       time step    [s]
      - t_max:    duration of calculated motion [s]

    Output: (x_arr, y_arr) [m]
        where x_arr and y_arr are NumPy arrays containing the x and y coordinates of
        the planet at each time step, starting with the initial position
    """
    ##################
    n_steps = round(t_max/dt)
    x, y = position
    x_arr, y_arr = [x], [y]
    
    # calculate position until time max is reached
    n = 0
    while n<n_steps:
        n += 1
        #calculate and append x and y coordinates to arrays
        position, velocity = move_planet(position, velocity, m_star, dt)
        x, y = position
        x_arr.append(x)
        y_arr.append(y)
   
    return (x_arr, y_arr)
    ##################

#### Testing your function

If you have implemented all three functions correctly, the following cell should show an almost circular orbit.

In [ ]:
G       = 6.6743e-11 # gravitational constant [m^3 kg^-1 s^-2]
YEAR    = 3.154e7    # one year in seconds
M_STAR  = 2.5e30     # mass of star [kg]

dt = YEAR/10000      # time step for Euler's method
pos = np.array([2.0e11, 0])    # initial position (x,y) [m]
vel = np.array([0,2.89e4])     # initial velocity (vx,vy) [m/s]

# Calculate trajectory
x_test, y_test = trajectory(pos,vel,M_STAR,dt,2*YEAR)

# Plot trajectory
fig, axes = plt.subplots()
axes.set_aspect('equal')
plt.title('Trajectory of planet')
plt.xlabel('x [m]')
plt.ylabel('y [m]')
plt.plot(x_test, y_test)

### 4. Animating the orbit

To better visualise the motion of the planet, we create an animation showing the changing position of the planet, with a trail to show its path.

As seen in unit 9, we define a function that updates the display of the planet and trail:-

In [ ]:
def animate_with_trail(i, planet, trail, x_arr, y_arr):
    """
    Update display for projectile motion, with trail.
    
    Arguments:
        i:      frame number (from 0 at time = 0)
        planet: Line2D object containing coordinates of planet
        trail:  Line2D object containing coordinates of trail
        x_arr:  array of x coordinate at teach time step
        y_arr:  array of y coordinate at teach time step

    Result: updates coordinates in Line2D objects provided as arguments
    "bodies" and "trails".
    """
    x, y = x_arr[i], y_arr[i]
    planet.set_data([x],[y])                # Body gets coordinates of current position
    trail.set_data(x_arr[:i+1],y_arr[:i+1]) # Trail has all points up to the current one

To make it easier to create different simulations without repeating too much code, we define a function to create an animation using the calculated trajectory. As well as the arrays of x and y values, we have to pass it the time step used in the simulation, the number of frames per second to use in the animation, and the "speedup" factor, which is the ratio of the simulated time (which is typically of order years) to the real time (typically of order seconds).

To create a relatively smoothly animation with the finite power of our computer, we can't plot every position we have calculated along the trajectory, but have to pick every $N$th position, where $N$ is chosen to match our animation frame rate to our calculated trajectory.

In [ ]:
def create_animation(x_arr, y_arr, dt, fps, speedup):
    """
    Create an animation of an object with given trajectory.
    
    Arguments:
        x_arr, y_arr: arrays of x and y coordinates at each time step [m]
        dt:           time step in simulation [s]
        fps:          number of frames per second for animation
        speedup:      ratio of simulated time to screen time
    """
    # Find range of coordinates to include full trajectory
    x_max = max(np.abs(x_arr))
    y_max = max(np.abs(y_arr))
    r_max = 1.05 * max(x_max, y_max)
    
    # Create and configure figure and axes
    plt.ioff()
    fig, axes = plt.subplots()
    axes.set_xlim(-r_max,r_max)
    axes.set_ylim(-r_max,r_max)
    axes.set_aspect('equal')
    axes.set_title('Trajectory of planet')
    axes.set_xlabel('x [m]')
    axes.set_ylabel('y [m]')

    # Create Line2D objects to represent body and trail
    planet,  = axes.plot([],[],'o')
    trail,   = axes.plot([],[],'-')

    # Get arrays of coordinates to display (subset of provided data)
    frame_interval = 1/fps                  # time between frames [s]
    skip = round(frame_interval*speedup/dt) # number of calculated points per frame
    x_plot = x_arr[::skip]  # take every Nth value (N = skip)
    y_plot = y_arr[::skip]
    n_frames = len(x_plot)

    # Create and return animation object
    ani = animation.FuncAnimation(fig,animate_with_trail, frames=n_frames, interval=frame_interval*1000,
                                  fargs=[planet, trail, x_plot, y_plot])
    return ani

***Animating your calculated orbit.***

In the cell below, write some code that calls this function to create an animation of the orbit you have calculated above. You should choose parameters to create a fairly smooth animation that takes a few seconds to run.

In [ ]:
##################
# max time: 2 years [s], speedup: year/2 [s], fps: 60 [s^-1]

xval, yval = trajectory(pos,vel,M_STAR,dt,2*YEAR)
ani = create_animation(xval, yval, dt, 60, YEAR/2)
HTML(ani.to_jshtml())
##################

### 5. Investigation

You should insert code and text cells below as required to investigate and discuss the effect of changing the parameters of the animation: time step, mass of star, initial position and velocity of planet.

## Adjusting Position:
For the next 3 animations I have adjusted the initial position of the planet to simulate different scenarios where the initial velocity is constant and exhibit the different orbits formed.

First I will start with the radius half the original radius:

In [ ]:
pos = (1e11, 0)    # initial position (x,y) [m]
xval, yval = trajectory(pos,vel,M_STAR,dt,2*YEAR)
ani1 = create_animation(xval, yval, dt, 60, YEAR/2)
HTML(ani1.to_jshtml())

In the first simulation, when the planet is closest to the star in its eliptical orbit, the approximation appears to deteriorate in smoothness. This is due to the speedup parameter equal to half a year. With a smaller value between simulated to real time, the animation will be more accurate but will take longer to calculate before running.

In [ ]:
pos = (1.0e11, 0) # initial position (x,y) [m]
xval, yval = trajectory(pos,vel,M_STAR,dt,2*YEAR)
ani2 = create_animation(xval, yval, dt, 30, YEAR/8) #set speedup parameter to smaller value
HTML(ani2.to_jshtml())

When the initial position of the planet is closer to the star with the same velocity it forms  elliptical orbits that spiral larger due to the initial velocity being too large for the initial position. Following this the next cell will look at how increasing the position affects the orbit of the planet.

In [ ]:
pos = (3e11, 0) # initial position (x,y) [m]
xval, yval = trajectory(pos,vel,M_STAR,dt,8*YEAR)
ani3 = create_animation(xval, yval, dt, 30, YEAR)
HTML(ani3.to_jshtml())

From this orbit we can conclude that the orbit also becomes elliptical, although with a larger semi-major axis. However let us continue increasing the position to possibly obtain a highly eccentric orbit, or see at what point the planet will be ejected from the star's sphere {or in this case circle) of influence.

In [ ]:
pos = (4e11, 0) # initial position (x,y) [m]
xval, yval = trajectory(pos,vel,M_STAR,dt,8*YEAR)
ani4 = create_animation(xval, yval, dt, 30, YEAR)
HTML(ani4.to_jshtml())

In this case, we cannot tell whether the orbit is bound , however if we increase the maximum time we can conclude that the orbit is unbound and has been ejected from the system.

In [ ]:
pos = (4e11, 0) # initial position (x,y) [m]
xval, yval = trajectory(pos,vel,M_STAR,dt,12*YEAR)
ani4 = create_animation(xval, yval, dt, 30, YEAR)
HTML(ani4.to_jshtml())

## Adjusting Velocity:

In the following cells I have adjusted the initial velocity of the orbiting planet, with the initial position constant at (2e11, 0) and the mass of the star constant. This is to demonstrate how the initial velocity affects the orbit of the planet.

In [ ]:
pos = (2.0e11, 0) # initial position (x,y) [m]
vel = (0, 1e4)    # initial velocity (vx,vy) [m/s]
xval, yval = trajectory(pos,vel,M_STAR,dt,6*YEAR)
ani5 = create_animation(xval, yval, dt, 30, YEAR/2)
HTML(ani5.to_jshtml())

In the cell above, the initial velocity is much smaller than the original (1/3 the magnitude). This causes the first complete orbit to have a high eccentricity, with the second orbit ejecting the planet out of the system. This a great example of how manipulating to a smaller velocity can further accelerate an object to faster velocities through the gravity of the star (or another massive celestial object). This simulation could be used to learn about gravitational slingshots [3].

In [ ]:
vel = (0, 4e4) # initial velocity (vx,vy) [m/s]
xval, yval = trajectory(pos,vel,M_STAR,dt,70*YEAR)
ani6 = create_animation(xval, yval, dt, 30, 8*YEAR)
HTML(ani6.to_jshtml())

The cell above shows the effect of using a larger initial veloity: the planet's orbit now becomes nearly 60 times larger. For the full orbit, the speedup ratio had to be increased to 8 years and the maximum time set to 70 years for the animation to be less than 10 seconds long.

In [ ]:
vel = (0, 3.5e4) # initial velocity (vx,vy) [m/s]
xval, yval = trajectory(pos,vel,M_STAR,dt,4*YEAR)
ani7 = create_animation(xval, yval, dt, 30, YEAR/2)
HTML(ani7.to_jshtml())

By adjusting the velocity to be 500 m/s slower, the orbit of the planet now is less than 4 years in duration, however it is still elliptical.

In [ ]:
vel = (0, 5e3) # initial velocity (vx,vy) [m/s]
xval, yval = trajectory(pos,vel,M_STAR,dt,YEAR)
ani8 = create_animation(xval, yval, dt, 30, YEAR/8)
HTML(ani8.to_jshtml())

The above animation demonstrates how when the velocity is much smaller than the velocity allowing it to form a near-circular orbit, the planet will immediately "fall" towards the star, with its small velocity allowing it to be ejected by the star out of the system rather than fall towards the origin and stay there. However if the simulation didn't use point-like particle to represent the planet and star, it would be highly likely that the planet would be engulfed by the star's surface as it passed very close to the centre of the star, thus affecting its actual orbit compared to this simulated orbit.

## Adjust the Mass of the Star:
Starting with the conditions from the intial orbit of the planet, I will adjust the mass of the star and view the effects on the orbit of the planet. Keeping the position and velocity the same as the first animation. In the following animation, the mass of the star will be doubled.

In [ ]:
pos = np.array([2.0e11, 0])    # initial position (x,y) [m]
vel = np.array([0,2.89e4])     # initial velocity (vx,vy) [m/s]
M_STAR  = 5e30     # mass of star [kg]
xval, yval = trajectory(pos,vel,M_STAR,dt,2*YEAR)
ani = create_animation(xval, yval, dt, 60, YEAR/2)
HTML(ani.to_jshtml())

Doubling the mass of the star doubles the force exerted on the planet, causing it to have similar behaviour as to when the radius of the planet was halved. The following animation will halve the mass of the star:

In [ ]:
M_STAR  = 1.25e30     # mass of star [kg]
xval, yval = trajectory(pos,vel,M_STAR,dt,2*YEAR)
ani = create_animation(xval, yval, dt, 60, YEAR/2)
HTML(ani.to_jshtml())

In this case the mass of the star is now too small to maintain the planet in a bound orbit, and the planet now leaves the system. The following animation quadruples the mass of the star:

In [ ]:
M_STAR  = 1e31     # mass of star [kg]
xval, yval = trajectory(pos,vel,M_STAR,dt,4*YEAR)
ani = create_animation(xval, yval, dt, 60, YEAR/2)
HTML(ani.to_jshtml())

In this animation we now observe how the planet cycles through several elliptical orbits of increasing eccentricity in order to find a stable orbit (or find a point of lowest potential where it will stay in a stable orbit).

## Conclusion

When analysing the initial parameters that would affect the simulation, increasing the mass of the star will linearly increase the force experienced by the planet, thus cause elliptical orbits, if the velocity of the planet did not exactly correspond with the appropriate radius of orbit, then the orbit will always form an ellipse or be an unbound orbit. Circular orbits only formed when the initial velocity and position of the planet were carefully selected and calculated using Kepler's Third Law [4]. Adjusting the time step (dt) had no significant effect on calculation time, the speedup parameter had the largest effect as it would determine how long the animation would be and thus how many frames would need to be calculated. As dt was small enough that the animation smoothness depended mostly on the speedup parameter this did not need to be changed to improve the smoothness of any of the animations.

## Part B

It is up to you to structure the rest of the notebook as you see fit, as you complete the tasks set in part B of the assignment.

You can call the functions you have defined in part A, or copy and adapt them in part B, but **DO NOT CHANGE THE CODE IN PART A SUCH THAT THE CELLS IN PART A NO LONGER WORK CORRECTLY.** You do not want to lose marks in section A in completing section B.

The following code has been taken from Reference [2]:

In [ ]:
# code taken from [2]
def animate_multiple(i, planets, trails, trajectories):
    """
    Update display for motion of multiple bodies with trails.
    
    Arguments:
        i:       frame number (from 0 at time = 0)
        planets: list of Line2D objects containing coordinates of each body to move
        trails:  list of Line2D objects containing coordinates of trails
        trajectories: list of trajectories, each of the form [x_arr, y_arr]
                 where x_arr and y_arr are arrays of x and y coordinates at
                 each time step
    
    Note that all trajectories must have the same number of points.
    """
    for j in range(len(trajectories)):    # for each trajectory ...
        trajectory = trajectories[j]
        x_arr, y_arr = trajectory         # get arrays of x and y coordinats
        x, y = x_arr[i], y_arr[i]
        planets[j].set_data([x],[y])
        trails[j].set_data(x_arr[:i+1],y_arr[:i+1]) # Trail has all points up to this one

In [ ]:
# code taken from [2]
def get_bodies_and_trails(axes, n):
    """
    Get the required number of Line2D objects to represent the bodies
    being animated, and the trails left behind them.
    
    Arguments:
        axes: the axes to be used for plotting the bodies
        n:    number of bodies in simulation
    
    Returns: (bodies, trails) where
        bodies is a list of Line2D objects to be plotted as discs
        trails is a list of Line2D objects to be plotted as lines
    """
    bodies = []
    trails = []
    for i in range(n):              # repeat n times
        body,  = axes.plot([],[],'o')           # Line2D with disc markers
        col    = body.get_color()               # get colour of body ...
        trail, = axes.plot([],[],'-',color=col) # and set trail (line) to same colour
        bodies.append(body)         # add Line2D objects to lists
        trails.append(trail)
    return bodies, trails

In [ ]:
# code taken from [2]
def create_animation_planets(trajectories, dt, fps, speedup):
    """
    Create an animation of objects with given trajectories.
    
    Arguments:
        trajectories: list of trajectories, each of the form [x_arr, y_arr]
                 where x_arr and y_arr are arrays of x and y coordinates at
                 each time step
        dt:           time step in simulation [s]
        fps:          number of frames per second for animation
        speedup:      ratio of simulated time to screen time
    """
    # Find range of coordinates to include full trajectories 
    r_max = 1.05 * np.max(np.abs(trajectories))

    # Create and configure figure and axes
    plt.ioff()
    fig, axes = plt.subplots()
    axes.set_xlim(-r_max,r_max)
    axes.set_ylim(-r_max,r_max)
    axes.set_aspect('equal')

    # Create Line2D objects to represent bodies and trails
    bodies, trails = get_bodies_and_trails(axes, 2)
    body = bodies[0]
    trail = trails[0]

    # Get arrays of coordinates to display (subset of provided data)
    frame_interval = 1/fps                  # time between frames [s]
    skip = round(frame_interval*speedup/dt) # number of calculated points per frame
    plot_trajectories = []
    for trajectory in trajectories:
        x_arr, y_arr = trajectory
        x_plot = x_arr[::skip]  # take every Nth value (N = skip)
        y_plot = y_arr[::skip]
        plot_trajectory = [x_plot, y_plot]
        plot_trajectories.append(plot_trajectory)
        n_frames = len(x_plot)
    
    # Create and return animation object
    ani = animation.FuncAnimation(fig,animate_multiple, frames=n_frames, interval=frame_interval*1000,
                                  fargs=[bodies, trails, plot_trajectories])
    return ani

The following code cell will simulate two planets orbiting the star. However, they do not interact with each other in that they do not exert a gravitational force on each other.

In [ ]:
# code adapted from [2]
# Constants and parameters
G       = 6.6743e-11 # gravitational constant [m^3 kg^-1 s^-2]
YEAR    = 3.154e7    # one year in seconds
M_STAR  = 2.5e30     # mass of star [kg]
dt = YEAR/10000      # time step for Euler's method

# Initial position (x,y) [m] and velocity (vx,vy) [m/s] of each planet
pos_a = np.array([2e11, 0])
vel_a = np.array([0,2.89e4])
pos_b = np.array([1.56e11,1.56e11])
vel_b = np.array([-1.95e4,1.95e4])

# Calculate trajectories (lists of x and y coordinates) [m]
x_a, y_a = trajectory(pos_a,vel_a,M_STAR,dt,2*YEAR)
x_b, y_b = trajectory(pos_b,vel_b,M_STAR,dt,2*YEAR)

# Put the results into a list of trajectories, with each trajectory being
# a list containing the lists of x and y coordinates.
traj_a = [x_a, y_a]
traj_b = [x_b, y_b]
trajectories = [traj_a, traj_b]

# One year is represented by 2 seconds in the animation
speedup = YEAR/2

# Create and display the animation
ani = create_animation_planets(trajectories, dt, 60, speedup)
HTML(ani.to_jshtml())

In this scenario, both planets will orbit in near circular orbits around the star, unaffecting eachother's orbits. This simulation in effect is drawing 1 planet at a time with different initial conditions and animating both trajectories on the same graph.

## Interacting Planets:
Now that there are more than 2 bodies with mass, all 3 bodies will exert forces on each other. So the overall force experienced by a planet will be the sum of the forces experienced from the star and the other planet.
This can be shown in the following equation:
$$ \vec{F_{1}} = -\frac{GMm_1\vec{r}}{|\vec{r}|^3} -\frac{Gm_2m_1\vec{r}_{21}}{|\vec{r}_{21}|^3} $$
where $ \vec{F_1} $ is the Force on the first planet, $\vec{r}_{21}$ is the displacement vector between planet 1 and 2 while $ \vec {r}$ is the displacement between the star and the planet, $M$ is the mass of the star, $m_1$ is the mass of the first planet and $m_2$ the mass of the second. 
The acceleration thus will be the total force divided by the mass of the planet:
$$ \vec{a} = \frac{\vec{F}}{m} $$
With this information we can now adapt our previous move_planet function to now move both planets simultaneously and output both positions and velocities.

In [ ]:
def move_planets_grav(position1, velocity1, position2, velocity2, m_star, mass1, mass2, dt):
    """
    Calculate motion of 2 planets interacting in the gravitational field of a star with given mass
    at the origin, using Euler's method.
    
    Input:
      - position1: position vector of planet 1 at start of time step (NumPy array)
      - velocity1: velocity vector of planet 1 at start of time step (NumPy array)
      - position2: position vector of planet 2 at start of time step (NumPy array)
      - velocity2: velocity vector of planet 2 at start of time step (NumPy array)
      - m_star:    mass of star
      - mass1:     mass of planet 1
      - mass2:     mass of planet 2
      - dt:        time step
      
    Output: (position_new, velocity_new)
      - [pos_new1, vel_new1]: list of position vector (NumPy array) and velocity vector (NumPy array)
        of planet 1 at end of time step.
      - [pos_new2, vel_new2]: list of position vector (NumPy array) and velocity vector (NumPy array)
        of planet 2 at end of time step.
      
    Depends on:
      - force = function to calculate the gravitational force between two objects
    """
    ##################
    starpos = np.array([0,0])
    ForceOn1 = force(position1, starpos, mass1, m_star) + force(position1, position2, mass1, mass2)
    ForceOn2 = force(position2, starpos, mass2, m_star) + force(position2, position1, mass2, mass1)
    accel1 = ForceOn1/mass1
    accel2 = ForceOn2/mass2
    
    pos_new1 = position1 + np.array(velocity1) * dt
    vel_new1 = np.array(velocity1) + accel1 * dt
    pos_new2 = position2 +np.array(velocity2) * dt
    vel_new2 = np.array(velocity2) + accel2 * dt
    return [pos_new1, vel_new1], [pos_new2, vel_new2]
    ##################

Now that our function outputs more values than before we now must adapt our trajectory function to handle the extra planet calculation and thus calculate both trajectories of the planets at the same time to produce 2 coordinate arrays.

In [ ]:
def trajectories(position1, velocity1, position2, velocity2, m_star, mass1, mass2, dt, t_max):
    """
    Calculate trajectories of two planets, given starting positions and velocities, using Euler's method.
    
    Input:
      - position: position vector of planet at start of simulation [m] (NumPy array)
      - velocity: velocity vector of planet at start of simulation [m] (NumPy array)
      - m_star:   mass of star [kg]
      - dt:       time step    [s]
      - t_max:    duration of calculated motion [s]

    Output: (x1_arr, y1_arr) [m], (x2_arr, y2_arr) [m]
        where x1_arr, y1_arr, x2_arr, y2_arr are NumPy arrays containing the x and y coordinates of
        the planets at each time step, starting with the initial position for each planet.
    """
    ##################
    n_steps = round(t_max/dt)
    x1, y1 = position1
    x2, y2 = position2
    x1_arr, y1_arr = [x1], [y1]
    x2_arr, y2_arr = [x2], [y2]
    # calculate position until time max is reached
    n = 0
    while n<n_steps:
        n += 1
        #calculate and append x and y coordinates to arrays
        [position1, velocity1], [position2,velocity2] = move_planets_grav(position1, velocity1, position2, velocity2, m_star, mass1, mass2, dt)
        x1, y1 = position1
        x1_arr.append(x1)
        y1_arr.append(y1)
        x2, y2 = position2
        x2_arr.append(x2)
        y2_arr.append(y2)
    return (x1_arr, y1_arr), (x2_arr, y2_arr)
    ##################

First we need to define some initial conditions and constants for our animation, then we will simulate the exact conditions as when the planets didn't interact.

In [ ]:
# Constants and parameters
G       = 6.6743e-11 # gravitational constant [m^3 kg^-1 s^-2]
YEAR    = 3.154e7    # one year in seconds
M_STAR  = 2.5e30     # mass of star [kg]
dt = YEAR/10000      # time step for Euler's method
M_1 = 6e24           # mass of planet 1 [kg]
M_2 = 6e24           # mass of planet 2 [kg]
# Initial position (x,y) [m] and velocity (vx,vy) [m/s] of each planet
pos_a = np.array([2e11, 0])
vel_a = np.array([0,2.89e4])
pos_b = np.array([1.56e11,1.56e11])
vel_b = np.array([-1.95e4,1.95e4])


In [ ]:
# Calculate trajectories (lists of x and y coordinates) [m]
traj_a, traj_b = trajectories(pos_a,vel_a,pos_b, vel_b, M_STAR, M_1, M_2,dt,2*YEAR)

traj = [traj_a, traj_b]

# One year is represented by 2 seconds in the animation
speedup = YEAR/2

# Create and display the animation
ani = create_animation_planets(traj, dt, 60, speedup)
HTML(ani.to_jshtml())

When using exactly the same conditions as before, both planets seem to show the same behaviour, forming circular orbits without causing each other's orbits to be diverted. As such the following animation has increased the mass of one of the planets by 100 times.

In [ ]:
M_2 = 6e26           # mass of planet 2 [kg]
# Calculate trajectories (lists of x and y coordinates) [m]
traj_a, traj_b = trajectories(pos_a,vel_a,pos_b, vel_b, M_STAR, M_1, M_2,dt,4*YEAR)

traj = [traj_a, traj_b]

# One year is represented by 2 seconds in the animation
speedup = YEAR/2

# Create and display the animation
ani = create_animation_planets(traj, dt, 60, speedup)
HTML(ani.to_jshtml())

Now, the mass of the second planet has a large enough gravitational force to pull the first planet out of its orbit and eject the planet to a new elliptical orbit with a much larger orbital period.

## Simulating a moon orbiting a planet:
In the following section I experimented with different initial parameters to create a moon orbiting a planet as they both orbit a star.

In [ ]:
# Constants and parameters
G       = 6.6743e-11 # gravitational constant [m^3 kg^-1 s^-2]
YEAR    = 3.154e7    # one year in seconds
M_STAR  = 2e30     # mass of star [kg]
dt = YEAR/10000      # time step for Euler's method
M_1 = 6e24           # mass of planet 1 [kg]
M_2 = 1.1e16           # mass of planet 2 [kg]
# Initial position (x,y) [m] and velocity (vx,vy) [m/s] of each planet
pos_a = np.array([1.47e11, 0])
vel_a = np.array([0,3e4])
pos_b = np.array([1.47e11,4e9])
vel_b = np.array([-1.5,3e4])

In [ ]:
# Calculate trajectories (lists of x and y coordinates) [m]
traj_a, traj_b = trajectories(pos_a,vel_a,pos_b, vel_b, M_STAR, M_1, M_2,dt,2*YEAR)

traj = [traj_a, traj_b]

# One year is represented by 4 seconds in the animation
speedup = YEAR/4

# Create and display the animation
ani = create_animation_planets(traj, dt, 60, speedup)
HTML(ani.to_jshtml())

This first animation exhibits a moon orbiting at the same period around the planet as the orbital period of the planet around the star. It is not entirely clear whether the both are just orbiting the star and if one is acting as a moon.

In [ ]:
# Constants and parameters
G       = 6.6743e-11 # gravitational constant [m^3 kg^-1 s^-2]
YEAR    = 3.154e7    # one year in seconds
M_STAR  = 2e30     # mass of star [kg]
dt = YEAR/10000      # time step for Euler's method
M_1 = 6e26          # mass of planet 1 [kg]
M_2 = 7.34e22        # mass of planet 2 [kg]
# Initial position (x,y) [m] and velocity (vx,vy) [m/s] of each planet
pos_a = np.array([1.47e11, 0])
vel_a = np.array([0,3e4])
pos_b = np.array([1.469e11,2e10])
vel_b = np.array([-2e3,3e4])

In [ ]:
# Calculate trajectories (lists of x and y coordinates) [m]
traj_a, traj_b = trajectories(pos_a,vel_a,pos_b, vel_b, M_STAR, M_1, M_2,dt,2*YEAR)

traj = [traj_a, traj_b]

# One year is represented by 4 seconds in the animation
speedup = YEAR/4

# Create and display the animation
ani = create_animation_planets(traj, dt, 60, speedup)
HTML(ani.to_jshtml())

This second animation I used trialed multiple values for the initial velocity of the moon to eventually obtain a velocity that will lead to a period of orbit around the planet that is roughly 1/3 of the planet's orbital period around the star. In this case, the moon's mass is 10000 times smaller than the planets so that the gravitational force will have a minute effect on the planet's velocity. The following animation shows what happens with the same initial conditions but the moon now a tenth of the mass of the planet.

In [ ]:
# Constants and parameters
G       = 6.6743e-11 # gravitational constant [m^3 kg^-1 s^-2]
YEAR    = 3.154e7    # one year in seconds
M_STAR  = 2e30     # mass of star [kg]
dt = YEAR/10000      # time step for Euler's method
M_1 = 6e26          # mass of planet 1 [kg]
M_2 = 6e25        # mass of planet 2 [kg]
# Initial position (x,y) [m] and velocity (vx,vy) [m/s] of each planet
pos_a = np.array([1.47e11, 0])
vel_a = np.array([0,3e4])
pos_b = np.array([1.469e11, 2e10])
vel_b = np.array([-2e3,3e4])

In [ ]:
# Calculate trajectories (lists of x and y coordinates) [m]
traj_a, traj_b = trajectories(pos_a,vel_a,pos_b, vel_b, M_STAR, M_1, M_2,dt,2*YEAR)

traj = [traj_a, traj_b]

# One year is represented by 4 seconds in the animation
speedup = YEAR/4

# Create and display the animation
ani = create_animation_planets(traj, dt, 30, speedup)
HTML(ani.to_jshtml())

The change in mass of the moon now has a significant effect on the planet's orbit, which leads to the moon quickly escaping the sphere of influence of the planet and seperating to become its own planet that also orbits the star.

## Conclusion:
I was successful in creating a simulation of a moon-planet system orbiting a star, through carefully choosing a distance between the moon and the planet such that the moon was within the Hill sphere of the planet and had a small enough velocity so that it would not escape the orbit around the planet. Furthermore, the moon should be of a small enough size to maintain a stable orbit otherwise the gravitational influence of the moon will destabilise its own orbit as it affects the orbit of the planet around the star.

## References
[1] Chislett R, Dash L, Waugh B.*PHAS0007 Computing Final Assignment
2022-23: Simulating Planetary Orbits*. [online] UCL: London; 2022 [Accessed 29 December 2022]. Available from: https://moodle.ucl.ac.uk/mod/resource/view.php?id=4630493

[2] Waugh B. *PHAS0007 Computing final assignment: supplementary notebook*. [online] UCL: London; 2022 [Accessed 28 December 2022]. Available from: https://moodle.ucl.ac.uk/mod/resource/view.php?id=4629403

[3] Davis P. *Basics of Space Flight* [online] NASA: 2022 [Accessed 28 December 2022]. Available from: https://solarsystem.nasa.gov/basics/chapter4-1/

[4] Kepler J. *The harmony of the world.* American Philosophical Society; 1997. p. 411.  Available from:  https://books.google.co.uk/books?printsec=frontcover&vid=ISBN0871692090&redir_esc=y#v=onepage&q&f=false